In [1]:
library(fastDummies)
library(cplm)
library(tweedie)
library(statmod)
library(TDboost)
library(mgcv)
library(MASS)
library(glmnet)
source("S_U2.R")
options(warn=-1)

Loading required package: coda

Loading required package: Matrix

Loading required package: splines

Loading required package: lattice

Loaded TDboost 1.2


Loading required package: nlme


Attaching package: 'nlme'


The following objects are masked from 'package:cplm':

    fixef, ranef, VarCorr


This is mgcv 1.8-38. For overview type 'help("mgcv-package")'.

Loaded glmnet 4.1-2



In [2]:
D_beta<-matrix(nrow=16,ncol=100)
A_beta<-matrix(nrow=16,ncol=100)
LASSO_beta<-matrix(nrow=16,ncol=100)
B_beta<-matrix(nrow=16,ncol=100)
nc=1

In [3]:
while(nc<=100){
    sigma_matrix = matrix(nrow=5,ncol=5)
    for (i in 1:5){
        for (j in 1:5){
            if (i==j){
                sigma_matrix[i,j]<-1
            }
            else{
                sigma_matrix[i,j]<-0.5^abs(i-j)
            }
        }
    }
    x<- mvrnorm(n = 1000, mu=c(rep(0,5)), Sigma=sigma_matrix, tol = 1e-6, empirical = FALSE, EISPACK = FALSE)
    colnames(x)=c('x1','x2','x3','x4','x5')
    X<-as.data.frame(x)
    b_t<-c(-0.3,0.8,-0.8,0.8,0,0,-0.8,-0.8,-0.8)
    design_x <- model.matrix(~.+x1*x2+x1*x3+x2*x3,data=X)
    mu<-exp(design_x%*%b_t)
    y_sim<- rTweedie(mu,p=1.5,phi=5)
    dd<-as.data.frame(cbind(design_x[,2:6],y_sim))
    sam <- sample(rep(1:2,len=1000))
    sim_train<-dd[sam==1,]
    sim_test<-dd[sam==2,]
    sim_train<-as.data.frame(sim_train)
    sim_test<-as.data.frame(sim_test)
    colnames(sim_train)[6]='y'
    colnames(sim_test)[6]='y'
    fit_0<-glm(y~.^2,data=sim_train,family=tweedie(link.power=0,var.power=1.5),control=list(maxit=500))
    beta_0_t<-c(fit_0$coefficients[1:6])
    index=combn(c(1:5),2)
    int_coef<-fit_0$coefficients[-(1:6)]
    r_0_t<-c()
    for (i in 1:length(int_coef)){
        tmp_coef<-int_coef[i]/(beta_0_t[index[1,i]+1]*beta_0_t[index[2,i]+1])
        r_0_t<-c(r_0_t,tmp_coef)
    }
    X<-model.matrix(~.^2-1,data=sim_train[,1:5])
    qw=cv.glmnet(x=X,y=sim_train$y,family=tweedie(link.power=0,var.power=1.5),control=list(maxit=500),nlambda=10)
    LASSO_beta[,nc]<-coef(qw, s=qw$lambda.min)[,1]
    as=cv.glmnet(x=X[,1:5],y=sim_train$y,family=tweedie(link.power=0,var.power=1.5),control=list(maxit=500),nlambda=10)
    df=cv.glmnet(x=X[,-c(1:5)],y=sim_train$y,family=tweedie(link.power=0,var.power=1.5),control=list(maxit=500),nlambda=10,nfolds=5)
    r_lamb<-df$lambda
    beta_lamb<-as$lambda
    n_lamb<-length(beta_lamb)
    power=1.5
    final_result1=matrix(nrow=n_lamb,ncol=n_lamb)
    final_result2=matrix(nrow=n_lamb,ncol=n_lamb)
    final_result3=matrix(nrow=n_lamb,ncol=n_lamb)
    for (i in 1:n_lamb){
        for (j in 1:n_lamb){
            coef_result<-shim_lasso_update_2(sim_train[,1:5],sim_train$y,power,lamb_r=r_lamb[i],lamb_b=beta_lamb[j],r_0_t,beta_0_t)
            fitted<-ts_fitted(sim_test[,1:5],coef_result$'beta',coef_result$'r')
            final_result1[i,j]=mean(tweedie.dev(sim_test$y,fitted,1.5))
            fitted<-ts_fitted(sim_train[,1:5],coef_result$'beta',coef_result$'r')
            if (is.infinite(mean(tweedie.dev(sim_train$y,fitted,1.5),na.rm=T))){
                    final_result2[i,j]<-999999999
                    final_result3[i,j]<-999999999
                        }
            else{
                den<-dtweedie(sim_train$y,mu=fitted+0.0000001,phi=sum(tweedie.dev(sim_train$y,fitted,1.5),na.rm=T)/length(sim_train$y),power=1.5)
                df<-length(which(r_beta(coef_result$'beta',coef_result$'r')!=0))
                final_result2[i,j]<--2*sum( log(den))+2*(df)
                final_result3[i,j]<--2*sum( log(den))+log(length(sim_train$y))*(df)
                }
    }
        }
    index1<-which(final_result1==min(final_result1,na.rm=T), arr.ind = TRUE)[1,]
    index2<-which(final_result2==min(final_result2,na.rm=T), arr.ind = TRUE)[1,]
    index3<-which(final_result3==min(final_result3,na.rm=T), arr.ind = TRUE)[1,]
    coef_result<-shim_lasso_update_2(sim_train[,1:5],sim_train$y,power,lamb_r=r_lamb[index1[1]],lamb_b=beta_lamb[index1[2]],r_0_t,beta_0_t)
    D_beta[,nc]<-r_beta(coef_result$beta,coef_result$r)
    coef_result<-shim_lasso_update_2(sim_train[,1:5],sim_train$y,power,lamb_r=r_lamb[index2[1]],lamb_b=beta_lamb[index2[2]],r_0_t,beta_0_t)
    A_beta[,nc]<-r_beta(coef_result$beta,coef_result$r)
    coef_result<-shim_lasso_update_2(sim_train[,1:5],sim_train$y,power,lamb_r=r_lamb[index3[1]],lamb_b=beta_lamb[index3[2]],r_0_t,beta_0_t)
    B_beta[,nc]<-r_beta(coef_result$beta,coef_result$r)
    nc=nc+1
}

In [7]:
which(is.na(D_beta[1,]))

integer(0)

In [4]:
write.matrix(A_beta,file="simulation222Ac.csv")
write.matrix(D_beta,file="simulation222Dc.csv")
write.matrix(B_beta,file="simulation222Bc.csv")
write.matrix(LASSO_beta,file="simulation222Lc.csv")

In [4]:
dim(D_beta_1)

[1]  16 100

In [5]:
check_function<-function(x){
    xlen<-length(x)
    return(all(x[1:4]!=0)&all(x[5:6]==0)&all(x[7:8]!=0)&all(x[9:10]==0)&all(x[11]!=0)&all(x[12:xlen]==0))
}

check_function2<-function(x){
    xlen<-length(x)
    return(all(x[1:4]!=0)&all(x[7:8]!=0)&x[11]!=0)
}

distance_function<-function(x,b=c(-0.3,0.8,-0.8,0.8,0,0,-0.8,-0.8,0,0,-0.8,rep(0,5))){
    return(sqrt(mean((x-b)^2)))
}

distance_function_2<-function(x,b=c(-0.3,0.8,-0.8,0.8,0,0,-0.8,-0.8,0,0,-0.8,rep(0,5))){
    return(mean(abs(x-b)))
}

In [6]:
length(which(apply(A_beta,2,check_function)))
length(which(apply(B_beta,2,check_function)))
length(which(apply(D_beta,2,check_function)))
length(which(apply(LASSO_beta,2,check_function)))

[1] 4

[1] 23

[1] 2

[1] 0

In [8]:
length(which(apply(A_beta,2,check_function2)))
length(which(apply(B_beta,2,check_function2)))
length(which(apply(D_beta,2,check_function2)))
length(which(apply(LASSO_beta,2,check_function2)))

[1] 100

[1] 93

[1] 100

[1] 100

In [10]:
mean(apply(A_beta,2,distance_function_2))
mean(apply(B_beta,2,distance_function_2))
mean(apply(D_beta,2,distance_function_2))
mean(apply(LASSO_beta,2,distance_function_2))

[1] 0.09818407

[1] 0.09554936

[1] 0.09405343

[1] 0.1013817

In [11]:
length(which(apply(A_beta,2,check_function)))
length(which(apply(D_beta,2,check_function)))
length(which(apply(LASSO_beta,2,check_function)))

[1] 20

[1] 9

[1] 0

In [12]:
length(which(apply(A_beta,2,check_function_2)))
length(which(apply(D_beta,2,check_function_2)))
length(which(apply(LASSO_beta,2,check_function_2)))

[1] 100

[1] 98

[1] 100

In [13]:
mean(apply(A_beta,2,distance_function))
mean(apply(A_beta,2,distance_function_2))

[1] 0.169377

[1] 0.1118464

In [14]:
mean(apply(D_beta,2,distance_function))
mean(apply(D_beta,2,distance_function_2))

[1] 0.1667313

[1] 0.1128164

In [15]:
mean(apply(LASSO_beta,2,distance_function))
mean(apply(LASSO_beta,2,distance_function_2))

[1] 0.1404023

[1] 0.09964954

In [19]:
mean(apply(A_beta[,which(apply(A_beta,2,check_function_2))],2,distance_function_2))
mean(apply(D_beta[,which(apply(D_beta,2,check_function_2))],2,distance_function_2))
mean(apply(LASSO_beta[,which(apply(LASSO_beta,2,check_function_2))],2,distance_function_2))

[1] 0.1118464

[1] 0.1114061

[1] 0.09964954